In [ ]:
import shutil
import os

In [ ]:
import sys
sys.path.append(os.path.abspath("../framework"))
sys.path.append(os.path.abspath("../concrete"))

In [ ]:
from ConcAgent import ConcAgent
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from framework import TrainId, StoreField

In [ ]:
from workUtils import *

In [ ]:
trainLogFolderPath = "./trainLog"

In [ ]:
from glob import glob
import sqlite3
from datetime import datetime 
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

cur.execute("""

Create Table TrainLog
(
    buildOrderId Text, /* accord to a build order */
    trainId   Text, /* accord to a trained agent along with a simulation time*/
    timeSimulation Integer,
    timestamp timestamp
)
""")

concStore = ConcStore()
for filePath in glob(os.path.join(trainLogFolderPath, "*")):
    trainId = TrainId(os.path.basename(filePath))
    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(trainId)
    
    cur.execute('''
    Insert Into TrainLog (buildOrderId, trainId, timeSimulation, timestamp)
    values (?, ?, ?, ?)
    ''', (
        storeField.getBuildOrder().description
        , str(trainId)
        , storeField.getTimeSimulation()
        , datetime.strptime(storeField.getTimeStamp(), "%Y-%m-%d %H:%M:%S")
    ))    

In [ ]:
buildOrderIdGiven = "5iqyogSD3H9tEO1h"

In [ ]:
concStore = ConcStore()
agentFactory = ConcAgentFactory()

In [ ]:
cur.execute('''
Select
    count(*)
    From TrainLog
    Where buildOrderId = ?
    ''', (buildOrderIdGiven,))
assert cur.fetchone()[0] > 0, "NO DATA WITH THE GIVEN BUILD ID"
    

cur.execute('''
Select 
    trainId
    , timeSimulation
    , timestamp
    From TrainLog
    Where buildOrderId = ?
    Order By timeSimulation
    ''', (buildOrderIdGiven,))

for (trainIdStr, timeSimulation, timestamp) in cur.fetchall():

    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(TrainId(trainIdStr))
    
    # restore agent
    agent = agentFactory.create(storeField.getBuildOrder())
    agent.loadFromMemento(storeField.getAgentMemento())